# Taxas de rendimento no EM do RJ
---

Dados de Taxa de Rendimento (aprovação, reprovação e abandono) a nível de escola disponibilizados pelo [INEP](http://portal.inep.gov.br/web/guest/indicadores-educacionais), calculado através do Censo Escolar.

> *As taxas de rendimento escolar de cada instituição são geradas a partir da soma da quantidade de alunos aprovados, reprovados e que abandonaram a escola ao final de um ano letivo. Elas são importantes porque geram o Indicador de Rendimento, utilizado no cálculo do Ideb.*
> 
> *Para calcular as taxas de aprovação, reprovação e abandono, o Inep se baseia em informações sobre o movimento e o rendimento escolar dos alunos, fornecidas pelas escolas e pelas redes de ensino municipais, estaduais e federais. - [Taxas de Rendimento | Academia QEdu](https://academia.qedu.org.br/censo-escolar/taxa-de-rendimento/)*

## Resumo

- 24 (80%) das 30 escolas com maior reprovação e 22 (70%) das 30 escolas com maior abandono estão na cidade do Rio, embora a cidade concentre apenas 22% das escolas estaduais.

### Import required libraries

In [618]:
# IMPORT CONFIGs
import os
import sys
sys.path.insert(0, '../')
sys.path.insert(0, '../scripts')
from config import RAW_PATH, TREAT_PATH, OUTPUT_PATH


# DATA ANALYSIS & VIZ TOOLS
from copy import deepcopy
import yaml
from collections import Counter

import pandas as pd
import seaborn as sns
pd.options.display.max_columns = 999

%pylab inline
pylab.rcParams['figure.figsize'] = (12, 12)


# For changes in .py
%reload_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

Populating the interactive namespace from numpy and matplotlib


In [628]:
import scripts.load_inep_taxa_rendimento as taxa_rendimento

# Taxa de Rendimento de Escolas

URL : http://inep.gov.br/indicadores-educacionais

DOWNLOAD URL : http://download.inep.gov.br/informacoes_estatisticas/indicadores_educacionais/2018/TX_REND_ESCOLAS_2018.zip

In [437]:
column_maker = yaml.load('''
first:
    aprovacao : TAP
    reprovacao: TRE
    abandono: TAB
second:
    fundamental: 
        iniciais: F14
        finais: F58
        total: FUN
        1 ano: F00
        2 ano: F01
        3 ano: F02
        4 ano: F03
        5 ano: F04
        6 ano: F05
        7 ano: F06
        8 ano: F07
        9 ano: F08
    medio:
        total: MED
        seriado: MNS
        1 serie: M01
        2 serie: M02
        3 serie: M03
        4 serie: M04
extra:
    ano : ANO
    regiao : NO_REGIAO
    uf : SG_UF
    codigo do municipio : CO_MUNICIPIO
    nome do municipio : NO_MUNICIPIO
    nome da escola : NO_ENTIDADE
    codigo da escola : CO_ENTIDADE
    localizacao : TIPOLOCA
    dependencia administrativa: DEPENDAD
    rede: DEPENDAD
''')

In [633]:
config = yaml.load("""
tables :
    - year: 2018
      build_columns: 
        rows : [4, 5, 6]
        row_fill: [4, 5]
      drop_row: 7
    - year: 2017
      build_columns: 
        rows : [4, 5, 6]
        row_fill: [4, 5]
      drop_row: 7
    - year: 2016
      build_columns: 
        rows : [4, 5, 6]
        row_fill: [4, 5]
      drop_row: 7
    - year: 2015
      build_columns: 
        rows : [5, 6]
        row_fill: [5]
      drop_row: 7
    - year: 2014
      build_columns: 
        rows : [5, 6]
        row_fill: [5]
      drop_row: 7
    - year: 2013
      build_columns: 
        rows : [5, 6]
        row_fill: [5]
      drop_row: 7
    - year: 2012
      build_columns: 
        rows : [5, 6]
        row_fill: [5]
      drop_row: 7
    - year: 2011
      build_columns: 
        rows : [5, 6]
        row_fill: [5]
      drop_row: 7
      
    # Mudancas Necessarias
    - year: 2010
      build_columns: 
        rows : [5, 6]
        row_fill: [5]
      drop_row: 7

columns : ['ANO', 'CO_ENTIDADE', 'CO_MUNICIPIO', 'DEPENDAD', 'NO_ENTIDADE',
       'NO_MUNICIPIO', 'NO_REGIAO', 'SG_UF', 'TAB_F00', 'TAB_F01',
       'TAB_F02', 'TAB_F03', 'TAB_F04', 'TAB_F05', 'TAB_F06', 'TAB_F07',
       'TAB_F08', 'TAB_F14', 'TAB_F58', 'TAB_FUN', 'TAB_M01', 'TAB_M02',
       'TAB_M03', 'TAB_M04', 'TAB_MED', 'TAB_MNS', 'TAP_F00', 'TAP_F01',
       'TAP_F02', 'TAP_F03', 'TAP_F04', 'TAP_F05', 'TAP_F06', 'TAP_F07',
       'TAP_F08', 'TAP_F14', 'TAP_F58', 'TAP_FUN', 'TAP_M01', 'TAP_M02',
       'TAP_M03', 'TAP_M04', 'TAP_MED', 'TAP_MNS', 'TIPOLOCA', 'TRE_F00',
       'TRE_F01', 'TRE_F02', 'TRE_F03', 'TRE_F04', 'TRE_F05', 'TRE_F06',
       'TRE_F07', 'TRE_F08', 'TRE_F14', 'TRE_F58', 'TRE_FUN', 'TRE_M01',
       'TRE_M02', 'TRE_M03', 'TRE_M04', 'TRE_MED', 'TRE_MNS']
""")

In [615]:
year = 2011

In [617]:
df = pd.read_excel(RAW_PATH / f'TX_REND_ESCOLAS_{year}.xls')

In [619]:
df.tail(10)

,Unnamed: 0,Unnamed: 1,Ministério da Educação,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62
21864,2011,Norte,TO,1722107,XAMBIOA,Urbana,Municipal,17044790,ESCOLA MUNICIPAL OTACILIO CARDOSO,95.3,95.3,--,100,91.7,93.3,100,90,--,--,--,--,--,--,--,--,--,--,4.7,4.7,--,0,8.3,6.7,0,10,--,--,--,--,--,--,--,--,--,--,0,0,--,0,0,0,0,0,--,--,--,--,--,--,--,--,--,--
21865,2011,Norte,TO,1722107,XAMBIOA,Rural,Municipal,17046114,ESC MUNICIPAL GENIZA BERNARDO GRANGEIRO,100,100,--,100,100,100,100,100,--,--,--,--,--,--,--,--,--,--,0,0,--,0,0,0,0,0,--,--,--,--,--,--,--,--,--,--,0,0,--,0,0,0,0,0,--,--,--,--,--,--,--,--,--,--
21866,2011,Norte,TO,1722107,XAMBIOA,Rural,Municipal,17046157,ESCOLA MUNICIPAL PROFª MARIA DO CARMO PEREIRA,72.7,72.7,--,100,50,100,0,100,--,--,--,--,--,--,--,--,--,--,27.3,27.3,--,0,50,0,100,0,--,--,--,--,--,--,--,--,--,--,0,0,--,0,0,0,0,0,--,--,--,--,--,--,--,--,--,--
21867,2011,Norte,TO,1722107,XAMBIOA,Rural,Municipal,17050634,ESCOLA MUNICIPAL EUNICE MACEDO,73.7,73.7,--,100,40,66.7,100,--,--,--,--,--,--,--,--,--,--,--,26.3,26.3,--,0,60,33.3,0,--,--,--,--,--,--,--,--,--,--,--,0,0,--,0,0,0,0,--,--,--,--,--,--,--,--,--,--,--
21868,2011,Norte,TO,1722107,XAMBIOA,Urbana,Particular,17054117,CENTRO EDUCACIONAL FRANCISCA ANTUNES DE CARVALHO,100,100,--,100,100,100,100,100,--,--,--,--,--,--,--,--,--,--,0,0,--,0,0,0,0,0,--,--,--,--,--,--,--,--,--,--,0,0,--,0,0,0,0,0,--,--,--,--,--,--,--,--,--,--
21869,Fonte: MEC/INEP/DEED/CSI.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21870,"Notas: 1) Inclui as taxas de aprovação, r...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21871,2) Inclui as taxas do Ensino F...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21872,"3) ""taxas de rendimento escola...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21873,4) Escolas em Atividade.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [637]:
scripts.load_inep_taxa_rendimento.main(df, config, column_maker, year)

5
Empty columns:  set()
Columns with the same name:  set()


NameError: name 'df' is not defined

In [636]:
sort

<function numpy.sort>

### Schema BigQuery

,ANO,CO_ENTIDADE,CO_MUNICIPIO,DEPENDAD,NO_ENTIDADE,NO_MUNICIPIO,NO_REGIAO,SG_UF,TAB_F00,TAB_F01,TAB_F02,TAB_F03,TAB_F04,TAB_F04,TAB_F04,TAB_F04,TAB_F05,TAB_F06,TAB_F07,TAB_F08,TAB_F08,TAB_F08,TAB_F08,TAB_FUN,TAB_M01,TAB_M02,TAB_M03,TAB_M04,TAB_MED,TAB_MED,TAB_MED,TAB_MED,TAP_F00,TAP_F01,TAP_F02,TAP_F03,TAP_F04,TAP_F04,TAP_F04,TAP_F04,TAP_F05,TAP_F06,TAP_F07,TAP_F08,TAP_F08,TAP_F08,TAP_F08,TAP_FUN,TAP_M01,TAP_M02,TAP_M03,TAP_M04,TAP_MED,TAP_MED,TAP_MED,TAP_MED,TIPOLOCA,TRE_F00,TRE_F01,TRE_F02,TRE_F03,TRE_F04,TRE_F04,TRE_F04,TRE_F04,TRE_F05,TRE_F06,TRE_F07,TRE_F08,TRE_F08,TRE_F08,TRE_F08,TRE_FUN,TRE_M01,TRE_M02,TRE_M03,TRE_M04,TRE_MED,TRE_MED,TRE_MED,TRE_MED
0,2010,11022558.0,1100015.0,Estadual,EIEEF HAP BITT TUPARI,ALTA FLORESTA D'OESTE,Norte,RO,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,100.0,100.0,100.0,NaN,100.0,NaN,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rural,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010,11024291.0,1100015.0,Municipal,EMMEF 07 DE SETEMBRO,ALTA FLORESTA D'OESTE,Norte,RO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,100.0,83.3,80.0,100.0,90.5,100.0,90.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rural,0.0,0.0,16.7,20.0,0.0,9.5,0.0,9.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010,11024313.0,1100015.0,Municipal,EMMEF AFONSO AURELIO PORTO,ALTA FLORESTA D'OESTE,Norte,RO,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,100.0,NaN,100.0,100.0,100.0,100.0,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rural,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2010,11024348.0,1100015.0,Municipal,EMMEF ALUISIO DE AZEVEDO,ALTA FLORESTA D'OESTE,Norte,RO,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,NaN,NaN,66.7,100.0,90.0,100.0,90.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rural,0.0,NaN,NaN,33.3,0.0,10.0,0.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2010,11024372.0,1100015.0,Municipal,EMEIEF ANA NERY,ALTA FLORESTA D'OESTE,Norte,RO,0.0,0.0,0.0,8.3,0.0,1.7,0.0,1.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,100.0,85.7,91.7,87.5,93.2,87.5,93.2,56.0,66.7,81.3,88.9,69.4,88.9,69.4,81.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rural,0.0,0.0,14.3,0.0,12.5,5.1,12.5,5.1,44.0,33.3,18.7,11.1,30.6,11.1,30.6,18.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [115]:
print(':STRING,'.join(config['columns']))

ANO:STRING,CO_ENTIDADE:STRING,CO_MUNICIPIO:STRING,DEPENDAD:STRING,NO_ENTIDADE:STRING,NO_MUNICIPIO:STRING,NO_REGIAO:STRING,SG_UF:STRING,TAB_F00:STRING,TAB_F01:STRING,TAB_F02:STRING,TAB_F03:STRING,TAB_F04:STRING,TAB_F05:STRING,TAB_F06:STRING,TAB_F07:STRING,TAB_F08:STRING,TAB_F14:STRING,TAB_F58:STRING,TAB_FUN:STRING,TAB_M01:STRING,TAB_M02:STRING,TAB_M03:STRING,TAB_M04:STRING,TAB_MED:STRING,TAB_MNS:STRING,TAP_F00:STRING,TAP_F01:STRING,TAP_F02:STRING,TAP_F03:STRING,TAP_F04:STRING,TAP_F05:STRING,TAP_F06:STRING,TAP_F07:STRING,TAP_F08:STRING,TAP_F14:STRING,TAP_F58:STRING,TAP_FUN:STRING,TAP_M01:STRING,TAP_M02:STRING,TAP_M03:STRING,TAP_M04:STRING,TAP_MED:STRING,TAP_MNS:STRING,TIPOLOCA:STRING,TRE_F00:STRING,TRE_F01:STRING,TRE_F02:STRING,TRE_F03:STRING,TRE_F04:STRING,TRE_F05:STRING,TRE_F06:STRING,TRE_F07:STRING,TRE_F08:STRING,TRE_F14:STRING,TRE_F58:STRING,TRE_FUN:STRING,TRE_M01:STRING,TRE_M02:STRING,TRE_M03:STRING,TRE_M04:STRING,TRE_MED:STRING,TRE_MNS


In [439]:
new_columns

{'ano': 'ANO',
 'codigo da escola': 'CO_ENTIDADE',
 'codigo do municipio': 'CO_MUNICIPIO',
 'localizacao': 'TIPOLOCA',
 'nome da escola': 'NO_ENTIDADE',
 'nome do municipio': 'NO_MUNICIPIO',
 'rede': 'DEPENDAD',
 'regiao': 'NO_REGIAO',
 'taxa de abandono - ensino fundamental de 8 e 9 anos abandono - anos finais (6 ao 9 ano)': 'TAB_F58',
 'taxa de abandono - ensino fundamental de 8 e 9 anos abandono - anos iniciais (1 ao 5 ano)': 'TAB_F14',
 'taxa de abandono - ensino fundamental de 8 e 9 anos abandono no 1 ano ': 'TAB_F00',
 'taxa de abandono - ensino fundamental de 8 e 9 anos abandono no 2 ano': 'TAB_F01',
 'taxa de abandono - ensino fundamental de 8 e 9 anos abandono no 3 ano': 'TAB_F02',
 'taxa de abandono - ensino fundamental de 8 e 9 anos abandono no 4 ano': 'TAB_F03',
 'taxa de abandono - ensino fundamental de 8 e 9 anos abandono no 5 ano': 'TAB_F04',
 'taxa de abandono - ensino fundamental de 8 e 9 anos abandono no 6 ano': 'TAB_F05',
 'taxa de abandono - ensino fundamental de 8 